## QUBIT COUPLED TO A RESONATOR

 The first structure we want to analyze is the transmon qubit coupled to a $\lambda /2$ resonator, we'll consider the structure presented [**here**](https://qiskit-community.github.io/qiskit-metal/tut/4-Analysis/4.13-Analyze-transmon-and-resonator.html) and we'll compare the result obtained with QuLTRA and ANSYS

 **Note**: $\lambda/2$ resonators have an open end. To model this aspect on QuLTRA we terminated it on a very small capacitance


In [ ]:
import qultra as qu

C_pad_bot=(82.84141-32.48515-2.30760)*1e-15 #capacitance of the bottom pad respect to ground
C_pad_top=(93.38943-32.48515-19.67329)*1e-15 #capacitance of the top pad respect to ground
C_pads=32.48515e-15 #capacitance between the two pads
Cg_bot=2.30760e-15 #capacitance between the bottom pad and the coupling pad
Cg_top=19.67329e-15 #capacitance between the top pad and the coupling pad
Cg0=(60.14998-2.30760-19.67329)*1e-15 #capacitance between the coupling pad and ground
Lj=12.31e-9 #inductance of the Josephson junction

l=6e-3 #length of the CPW resonator

net=[qu.C(0,1,C_pad_bot),qu.C(2,0,C_pad_top),qu.C(1,2,C_pads),qu.C(1,3,Cg_bot),qu.C(2,3,Cg_top),qu.C(3,0,Cg0),qu.J(1,2,Lj),qu.CPW(4,3,l),qu.C(4,0,1e-25)]
qubit_resonator=qu.QCircuit(net,5,10)

qubit_resonator.show_all()

+------+------------+----------+
| Mode | Freq [GHz] | k [MHz]  |
+------+------------+----------+
|  1   |  5.85e+00  | 0.00e+00 |
|  2   |  9.35e+00  | 0.00e+00 |
+------+------------+----------+
Chi matrix [MHz]:
+------+----------+----------+
| Mode |    1     |    2     |
+------+----------+----------+
|  1   | 3.21e+02 | 1.93e+00 |
|  2   | 1.93e+00 | 2.90e-03 |
+------+----------+----------+


In [2]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

In [3]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'


gui = MetalGUI(design)

In [4]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um',
    pocket_height = '650um',
     hfss_inductance='12.31nH',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))
otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='1.75mm',  pos_y='0um', orientation='0'))
readout = RouteMeander(design, 'readout',  Dict(
            total_length='6 mm',
            hfss_wire_bonds = True,
            fillet='90 um',
            lead = dict(start_straight='100um'),
            pin_inputs=Dict(
                start_pin=Dict(component='Q1', pin='readout'),
                end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()

In [5]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb = EPRanalysis(design, "hfss")

In [6]:
eig_qb.sim.setup.max_passes = 10
eig_qb.sim.setup.max_delta_f = 0.1
eig_qb.sim.setup.min_freq_ghz=1
eig_qb.sim.setup.n_modes = 2
eig_qb.sim.setup.vars = Dict(Lj= '12.31 nH', Cj= '0 fF') 

eig_qb.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 1,
 'n_modes': 2,
 'max_delta_f': 0.1,
 'max_passes': 10,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '12.31 nH', 'Cj': '0 fF'}}

In [8]:
eig_qb.sim.run(components=['Q1','readout'], open_terminations=[('readout','end')],port_list=[])

INFO 04:11PM [connect_design]: 	Opened active design
	Design:    Design_hfss [Solution type: Eigenmode]
INFO 04:11PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:11PM [analyze]: Analyzing setup Setup
04:13PM 43s INFO [get_f_convergence]: Saved convergences to c:\Users\simona.zaccaria4\Desktop\hfss_eig_f_convergence.csv


In [12]:
eig_qb.setup.junctions.jj = Dict(rect='JJ_rect_Lj_Q1_rect_jj', line='JJ_Lj_Q1_rect_jj_',
                  Lj_variable='Lj', Cj_variable='Cj')
eig_qb.setup

{'junctions': {'jj': {'rect': 'JJ_rect_Lj_Q1_rect_jj',
   'line': 'JJ_Lj_Q1_rect_jj_',
   'Lj_variable': 'Lj',
   'Cj_variable': 'Cj'}},
 'dissipatives': {'dielectrics_bulk': ['main']},
 'cos_trunc': 8,
 'fock_trunc': 7,
 'sweep_variable': 'Lj'}

In [13]:
eig_qb.run_epr()


Design "Design_hfss" info:
	# eigenmodes    2
	# variations    1
Design "Design_hfss" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 1.85420701261036e-24
        energy_elec_substrate = 1.69917553337127e-24
        EPR of substrate = 91.6%

        energy_mag    = 1.85068391186877e-24
        energy_mag % of energy_elec_all  = 99.8%
        

Variation 0  [1/1]

  Mode 0 at 5.64 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.4%  2.139e-24 1.379e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 97.00%
	jj              0.992592  (+)        0.0307052
		(U_tot_cap-U_tot_ind)/mean=1.56%
Calculating Qdielectric_main for mode 0 (0/1)
p_dielectric_main_0 = 0.9194945926377162

  Mode 1 at 9.34 GHz   [2/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ

WARNING 04:41PM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project37\Design_hfss\2025-07-17 16-40-56.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization
Pm_norm=
modes
0    1.031801
1    1.041584
dtype: float64

Pm_norm idx =
      jj
0   True
1  False


  result['Q_coupling'] = self.Qm_coupling[variation][self.Qm_coupling[variation].columns[junctions]][modes]#TODO change the columns to junctions

  result['Qs'] = self.Qs[variation][self.PM[variation].columns[junctions]][modes] #TODO change the columns to junctions



*** P (participation matrix, not normlz.)
         jj
0  0.963022
1  0.001903

*** S (sign-bit matrix)
   s_jj
0     1
1    -1
*** P (participation matrix, normalized.)
      0.99
    0.0019

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       296     1.88
      1.88  0.00298

*** Chi matrix ND (MHz) 
       333     1.69
      1.69  0.00238

*** Frequencies O1 PT (MHz)
0    5344.782102
1    9343.662915
dtype: float64

*** Frequencies ND (MHz)
0    5326.965047
1    9343.679300
dtype: float64

*** Q_coupling
Empty DataFrame
Columns: []
Index: [0, 1]


...

###### Numerical diagonalization

Lj,12.31
0,5326.97
1,9343.68


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0         1
Lj                       
12.31 0  332.93  1.69e+00
      1    1.69  2.38e-03